In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 98.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [ ]:
import keras
import keras_nlp

import tensorflow as tf

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...


In [ ]:
model = gemma_lm.backbone
export_archive = keras.export.ExportArchive()
export_archive.track(model)
export_archive.add_endpoint(
    name="serve",
    fn=model.call,
    input_signature=[{
        "token_ids": tf.TensorSpec(shape=(None, 3), dtype=tf.float32),
        "padding_mask": tf.TensorSpec(shape=(None, 3), dtype=tf.float32),
    }],
)
export_archive.write_out('gemma_2')

Saved artifact at 'gemma_2'. The following endpoints are available:

* Endpoint 'serve'
  inputs (POSITIONAL_OR_KEYWORD): Dict[['token_ids', TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)], ['padding_mask', TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)]]
  training (POSITIONAL_OR_KEYWORD): Literal[None]
  mask (POSITIONAL_OR_KEYWORD): Literal[None]
Output Type:
  TensorSpec(shape=(None, 3, 2048), dtype=tf.float32, name=None)
Captures:
  135753328975312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135753328980416: TensorSpec(shape=(), dtype=tf.float32, name=None)
  135753329158528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135753329148144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135753329155888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135753329160464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135753329162048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135753329234640: TensorSpec(shape=(), dtype=tf.re

In [ ]:
tflite_model = tf.lite.TFLiteConverter.from_saved_model('gemma_2/').convert()

with open('gemma_2.tflite', 'wb') as f:
  f.write(tflite_model)

##Generate Gemma2 quantgized model

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('gemma_2/')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open('gemma2-quantized.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('gemma_2/')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

with open('gemma_2_fp16.tflite', 'wb') as f:
  f.write(tflite_model)